In [82]:
import numpy as np
import random 
import math
import copy

In [100]:
def fitness(x1, x2) : 
  return math.cos(math.radians(x1)) * math.sin(math.radians(x2)) - x1 / ((x2)**2 + 1)

class Individu() : 
  def __init__(self, function, c_len, r1, r2) : 
    self.function = function
    self.r1 = r1 
    self.r2 = r2
    self.c_len = c_len 

    self.chromosome = np.random.randint(2, size=c_len)
    self.x1 = self.decode(self.chromosome[0:len(self.chromosome)//2], r1[0], r1[1])
    self.x2 = self.decode(self.chromosome[len(self.chromosome)//2 :], r2[0], r2[1])
    self.fitness = function(self.x1, self.x2)

  def decode(self, chromosome, rmin, rmax) :
    n = len(chromosome)
    sig = 0
    sum = 0
    for i, gen in enumerate(chromosome): 
      sig += math.pow(2, -(i+1))
      sum += gen * math.pow(2, -(i+1))

    return rmin + ((rmax - rmin)/sig * sum)

  def updateGen(self, new) : 
    self.chromosome = new
    self.x1 = self.decode(self.chromosome[0:len(self.chromosome)//2], self.r1[0], self.r1[1])
    self.x2 = self.decode(self.chromosome[len(self.chromosome)//2 :], self.r2[0], self.r2[1])
    self.fitness = self.function(self.x1, self.x2)

  def setProbability(self, total) : 
    self.probability = 1 - (self.fitness / total)
  
class Populasi() : 
  def __init__(self, size, function, c_len, r1, r2) :
    self.size = size 
    self.function = function
    self.individu = self._initPopulation(size, function, c_len, r1, r2)
    self.individu = sorted(self.individu, key=lambda x: x.fitness, reverse=False)
    self.fittest = self.getFittest() 
    self.leastFittest = self.getLeastFittest()
    self.secondLeastFittest = self.getSecondLeastFittest()
    self.averageFitness = self.getAverageFitness()
  
  def _initPopulation(self, size, function, c_len, r1, r2) :
    pop = []
    for i in range(size) : 
      individu = Individu(function, c_len, r1, r2)
      pop.append(individu)
    return np.array(pop)

  def getFittest(self) : 
    return self.individu[0]

  def getLeastFittest(self) : 
    return self.individu[-1]

  def getSecondLeastFittest(self) : 
    return self.individu[-2]

  def getAverageFitness(self) : 
    total = 0
    for individu in self.individu : 
      total += individu.fitness 

    return total/self.size  


  def updateState(self) : 
    self.individu = sorted(self.individu, key=lambda x: x.fitness, reverse=False)
    self.leastFittest = self.getLeastFittest() 
    self.secondLeastFittest = self.getSecondLeastFittest()
    self.fittest = self.getFittest()
    self.averageFitness = self.getAverageFitness()

class GADriver() : 
  def __init__(self, population, pc, pm) : 
    self.population = population
    self._pc = pc
    self._pm = pm


  def config(self, pc, pm) : 
    self._pc = pc 
    self._pm = pm 

  def roulette(self) : 
    total = 0 
    for individu in self.population.individu : 
      total += individu.fitness 

    for individu in self.population.individu : 
      individu.setProbability(total)

    choice = [x for x in self.population.individu]
    prob   = [x.probability for x in self.population.individu ]

    self.parent = copy.copy(random.choices(choice, prob)[0])
    self.secondParent = copy.copy(random.choices(choice, prob)[0])

  def crossover(self) : 
    
    c1 = copy.copy(self.parent.chromosome)
    c2 = copy.copy(self.secondParent.chromosome)

    rand = random.choices([0, 1], [(1 - self._pc), self._pc])[0]
    if rand == 1 :
      random_point = np.random.randint(low=0, high=self.population.fittest.c_len) 

      l_fittest = c1[0:random_point]
      r_fittest = c1[random_point:]

      l_second = c2[0:random_point]
      r_second = c2[random_point:]
      
      c1 = np.append(l_fittest, r_second)
      c2 = np.append(r_fittest, l_second)

    return c1, c2

  def mutation(self, c1, c2) :

    for i in range(self.population.fittest.c_len) :
      rand = random.choices([0, 1], [(1 - self._pm), self._pm])[0]
      if rand == 1 : 
        if c1[i] == 0 : 
          c1[i] = 1 
        else : 
          c1[i] = 0

        if c2[i] == 0 : 
          c2[i] = 1 
        else : 
          c2[i] = 0

    return c1, c2

  def train(self, n_gen) :

    fit_history = []
    avg_history = []
    x1_history  = []
    x2_history  = []

    it = 1
    for i in range(n_gen) :

      fit_history.append(self.population.fittest.fitness)
      avg_history.append(self.population.averageFitness)
      x1_history.append(self.population.fittest.x1)
      x2_history.append(self.population.fittest.x2)


      print("Gen-{}".format(it))
      print("-"*10)
      print("Fittest")
      print("Chromosome : ", self.population.fittest.chromosome)
      print("x1, x2 : ", [self.population.fittest.x1, self.population.fittest.x2])
      print("Fitness    : ", self.population.fittest.fitness)
      print("")


      self.roulette()

      offspring1, offspring2 = self.crossover()
      offspring1, offspring2 = self.mutation(offspring1, offspring2) 
    
      self.population.leastFittest.updateGen(offspring1)
      self.population.secondLeastFittest.updateGen(offspring2)
      self.population.updateState()

      it+=1 

    return fit_history, x1_history, x2_history, avg_history


In [101]:
random.seed(42)

pc = 0.7
pm = 0.01
n_gen = 100
population = Populasi(8, fitness, 10, [-1, 1], [-1, 2])
driver = GADriver(population, pc, pm)

In [102]:
f_history, x1, x2, avg_history = driver.train(n_gen)

Gen-1
----------
Fittest
Chromosome :  [1 1 1 0 0 0 0 0 0 1]
x1, x2 :  [0.8064516129032258, -0.9032258064516129]
Fitness    :  -0.45988812432677384

Gen-2
----------
Fittest
Chromosome :  [1 1 1 0 0 0 0 0 0 1]
x1, x2 :  [0.8064516129032258, -0.9032258064516129]
Fitness    :  -0.45988812432677384

Gen-3
----------
Fittest
Chromosome :  [1 1 1 0 0 0 0 0 0 1]
x1, x2 :  [0.8064516129032258, -0.9032258064516129]
Fitness    :  -0.45988812432677384

Gen-4
----------
Fittest
Chromosome :  [1 1 1 0 0 0 0 0 0 1]
x1, x2 :  [0.8064516129032258, -0.9032258064516129]
Fitness    :  -0.45988812432677384

Gen-5
----------
Fittest
Chromosome :  [1 1 1 0 0 0 0 0 0 1]
x1, x2 :  [0.8064516129032258, -0.9032258064516129]
Fitness    :  -0.45988812432677384

Gen-6
----------
Fittest
Chromosome :  [1 1 1 0 0 0 0 0 0 1]
x1, x2 :  [0.8064516129032258, -0.9032258064516129]
Fitness    :  -0.45988812432677384

Gen-7
----------
Fittest
Chromosome :  [1 1 1 0 0 0 0 0 0 1]
x1, x2 :  [0.8064516129032258, -0.90322580645

In [103]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()

fig.add_trace(go.Scatter(
    x = [x for x in range(100)],
    y = f_history,
    name='Fittest'
))

fig.add_trace(go.Scatter(
    x = [x for x in range(100)],
    y = avg_history,
    name='Average Fitness'
))

fig.update_layout(
    title='Genetic Algorithm Descent', 
    autosize=False, template='plotly_dark',
    showlegend=True,

    xaxis_title='n-th Generation',
    yaxis_title='Fitness',
)

fig.show()


In [80]:
x = np.linspace(-1, 1, num=100)
y = np.linspace(-1, 2, num=100)
xs, ys = np.meshgrid(x, y)
zs = np.cos(np.radians(xs)) * np.sin(np.radians(ys)) - xs / ((ys)**2 + 1)

In [81]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()

fig.add_trace(go.Surface(
    x = x,
    y = y,
    z = zs,
))

fig.add_trace(go.Scatter3d(
    x=x1,
    y=x2, 
    z=f_history, 
    name="Fitness Step",
    mode='markers'

))

fig.update_layout(
    title='Genetic Algorithm Descent', 
    autosize=False, template='plotly_dark',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1), 
    scene = dict(
        xaxis_title='x1 values',
        yaxis_title='x2 values',
        zaxis_title='Fitness'),



)
fig.show()

In [89]:
fig = go.Figure(data =
    go.Contour(
        x=x,
        y=y,
        z=zs
    ))
fig.show()